### Debt Repayment Plan

#### Basic Case

- Data
    - **N<sub>i</sub>**  : Monthly income during period **i**
    - **E<sub>i</sub>**  : Monthly expense during period **i**
    - **M<sub>i</sub>**  : Monthly required savings during period **i**
    - **I**              : Interest paid on loan
    

- Decision Variables
    - **S<sub>i</sub>**  : Monthly savings during period **i**
    - **R<sub>i</sub>**  : Debt paid during month **i**
    - **X<sub>i</sub>**  : Debt owed at the beginning of month **i**

In [1]:
using JuMP
import HiGHS

In [2]:
# Data
debt = Dict(
    "Loan" => 10000,
    "Period" => 10,
    "Interest" => 6.8 / 100
)
periods = 1:debt["Period"]
const NET_INCOME = 4500

4500

In [3]:
# Initialize model
model = Model(HiGHS.Optimizer);

#### Decision Variables

In [4]:
# Monthly savings
@variable(model, 0 <= S[periods] <= NET_INCOME);

# Monthly debt reimbursed
@variable(model, 0 <= R[periods] <= NET_INCOME);

# Monthly balance
@variable(model, X[cat(periods, periods[end] + 1, dims = 1)] >= 0);

#### Constraints

In [5]:
# Debt owed at the beginning of period 1
@show @constraint(model,
    X[1] == debt["Loan"]
)

#= In[5]:2 =# @constraint(model, X[1] == debt["Loan"]) = X[1] == 10000.0


X[1] == 10000.0

In [6]:
# Ensure that all debt is paid off by the grace period
@show @constraint(model,
    X[debt["Period"]] == 0
)

#= In[6]:2 =# @constraint(model, X[debt["Period"]] == 0) = X[10] == 0.0


X[10] == 0.0

In [7]:
# Debt owed the beginning of period 2...end
@show @constraint(model,
    [i in periods],
    X[i + 1] == (1 + debt["Interest"]) * (X[i] - R[i])
)

#= In[7]:2 =# @constraint(model, [i in periods], X[i + 1] == (1 + debt["Interest"]) * (X[i] - R[i])) = 1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 1.068 R[1] - 1.068 X[1] + X[2] == 0.0
 1.068 R[2] - 1.068 X[2] + X[3] == 0.0
 1.068 R[3] - 1.068 X[3] + X[4] == 0.0
 1.068 R[4] - 1.068 X[4] + X[5] == 0.0
 1.068 R[5] - 1.068 X[5] + X[6] == 0.0
 1.068 R[6] - 1.068 X[6] + X[7] == 0.0
 1.068 R[7] - 1.068 X[7] + X[8] == 0.0
 1.068 R[8] - 1.068 X[8] + X[9] == 0.0
 1.068 R[9] - 1.068 X[9] + X[10] == 0.0
 1.068 R[10] - 1.068 X[10] + X[11] == 0.0


1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 1.068 R[1] - 1.068 X[1] + X[2] == 0.0
 1.068 R[2] - 1.068 X[2] + X[3] == 0.0
 1.068 R[3] - 1.068 X[3] + X[4] == 0.0
 1.068 R[4] - 1.068 X[4] + X[5] == 0.0
 1.068 R[5] - 1.068 X[5] + X[6] == 0.0
 1.068 R[6] - 1.068 X[6] + X[7] == 0.0
 1.068 R[7] - 1.068 X[7] + X[8] == 0.0
 1.068 R[8] - 1.068 X[8] + X[9] == 0.0
 1.068 R[9] - 1.068 X[9] + X[10] == 0.0
 1.068 R[10] - 1.068 X[10] + X[11] == 0.0

In [8]:
# Monthly expense can't exceed net income
@constraint(model,
    [i in periods],
    S[i] + R[i] <= NET_INCOME
)

1-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape},1,...} with index sets:
    Dimension 1, 1:10
And data, a 10-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.LessThan{Float64}}, ScalarShape}}:
 S[1] + R[1] <= 4500.0
 S[2] + R[2] <= 4500.0
 S[3] + R[3] <= 4500.0
 S[4] + R[4] <= 4500.0
 S[5] + R[5] <= 4500.0
 S[6] + R[6] <= 4500.0
 S[7] + R[7] <= 4500.0
 S[8] + R[8] <= 4500.0
 S[9] + R[9] <= 4500.0
 S[10] + R[10] <= 4500.0

#### Objective Function

In [9]:
# Objective function
@objective(model, Max, sum(S))

S[1] + S[2] + S[3] + S[4] + S[5] + S[6] + S[7] + S[8] + S[9] + S[10]

#### Solve Model

In [10]:
print(model)

In [11]:
optimize!(model)

Presolving model
16 rows, 24 cols, 39 nonzeros
10 rows, 18 cols, 27 nonzeros
10 rows, 18 cols, 27 nonzeros
Presolve : Reductions: rows 10(-12); columns 18(-13); elements 27(-25)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Ph1: 0(0) 0s
         11    -3.4532568000e+04 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 11
Objective value     :  3.4532568000e+04
HiGHS run time      :          0.00


In [12]:
@show termination_status(model)

termination_status(model) = MathOptInterface.OPTIMAL


OPTIMAL::TerminationStatusCode = 1

In [13]:
@show primal_status(model)

primal_status(model) = MathOptInterface.FEASIBLE_POINT


FEASIBLE_POINT::ResultStatusCode = 1

In [14]:
@show dual_status(model)

dual_status(model) = MathOptInterface.FEASIBLE_POINT


FEASIBLE_POINT::ResultStatusCode = 1

In [15]:
@show objective_value(model)

objective_value(model) = 34532.568


34532.568

In [16]:
function solve_infeasible(model)
    optimize!(model)
    if termination_status(model) != OPTIMAL
        @warn("The model was not solved correctly.")
        return nothing
    elseif termination_status(model) == FEASIBLE_POINT
        return objective_value(model)
    end
end

solve_infeasible(model)

Solving LP without presolve or with basis
Model   status      : Optimal
Objective value     :  3.4532568000e+04
HiGHS run time      :          0.00


In [17]:
using Printf
function print_payment_plan(model)
    @printf("| %-10s | %-15s | %-10s | %-10s | %-15s | %-10s | %-12s |\n",
        "Period", "Initial Balance", "Payment", "Savings", 
        "Int. Rate (%)", "Interest", "End Balance"
    )
    for i in periods
        @printf("| %10d | %15.2f | %10.2f | %10.2f | %15.2f | %10.2f | %12.2f |\n", 
            i,
            value(X[i]), 
            value(R[i]), 
            value(S[i]),
            debt["Interest"],
            if i == 1 0 else value(X[i]) - (value(X[i - 1] ) - value(R[i - 1])) end,
            value(X[i]) - value(R[i])
        )
    end
end;

In [18]:
print_payment_plan(model)

| Period     | Initial Balance | Payment    | Savings    | Int. Rate (%)   | Interest   | End Balance  |
|          1 |        10000.00 |    4500.00 |      -0.00 |            0.07 |       0.00 |      5500.00 |
|          2 |         5874.00 |    4500.00 |       0.00 |            0.07 |     374.00 |      1374.00 |
|          3 |         1467.43 |    1467.43 |    3032.57 |            0.07 |      93.43 |         0.00 |
|          4 |           -0.00 |      -0.00 |    4500.00 |            0.07 |      -0.00 |         0.00 |
|          5 |           -0.00 |      -0.00 |    4500.00 |            0.07 |      -0.00 |         0.00 |
|          6 |           -0.00 |      -0.00 |    4500.00 |            0.07 |      -0.00 |         0.00 |
|          7 |           -0.00 |      -0.00 |    4500.00 |            0.07 |      -0.00 |         0.00 |
|          8 |           -0.00 |      -0.00 |    4500.00 |            0.07 |      -0.00 |         0.00 |
|          9 |           -0.00 |      -0.00 |    4500.0

In [19]:
@printf("%119s", '\U2111')

                                                                                                                      ℑ